# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key
import warnings
warnings.filterwarnings("ignore")

In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [20]:
#%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
mapPlot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles= "OSM",
    frame_width = 800,
    frame_height = 600,
    color = "City",
    size = "Humidity"

)

mapPlot
# Display the map
# YOUR CODE HERE


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [14]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
idealTempDf = city_data_df.loc[(city_data_df["Max Temp"] >= 21) & (city_data_df["Humidity"]<=60)]
# Drop any rows with null values
# YOUR CODE HERE
idealTempDfCleaned = idealTempDf.dropna(how='any')
# Display sample data
# YOUR CODE HERE
idealTempDfCleaned

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
12,12,trairi,-3.2778,-39.2689,31.07,51,34,8.09,BR,1666108038
22,22,guiratinga,-16.3494,-53.7583,38.35,14,22,3.10,BR,1666108243
29,29,saint-louis,16.3333,-15.0000,38.80,24,4,1.78,SN,1666108247
34,34,naberera,-4.2000,36.9333,21.12,54,94,6.38,TZ,1666108252
...,...,...,...,...,...,...,...,...,...,...
536,536,nouadhibou,20.9310,-17.0347,33.99,27,49,9.26,MR,1666108546
542,542,bilma,18.6853,12.9164,31.90,18,0,4.17,NE,1666108548
545,545,kalabo,-14.9700,22.6814,33.55,14,100,2.90,ZM,1666108550
550,550,pali,25.7667,73.3333,27.18,26,0,1.38,IN,1666108552


### Step 3: Create a new DataFrame called `hotel_df`.

In [15]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = idealTempDfCleaned.copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df = hotel_df[["City","Country","Lat", "Lng", "Humidity"]]

hotel_df["Hotel Name"] = ''
# Display sample data
# YOUR CODE HERE
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,
12,trairi,BR,-3.2778,-39.2689,51,
22,guiratinga,BR,-16.3494,-53.7583,14,
29,saint-louis,SN,16.3333,-15.0000,24,
34,naberera,TZ,-4.2000,36.9333,54,
...,...,...,...,...,...,...
536,nouadhibou,MR,20.9310,-17.0347,27,
542,bilma,NE,18.6853,12.9164,18,
545,kalabo,ZM,-14.9700,22.6814,14,
550,pali,IN,25.7667,73.3333,26,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [16]:
# Set parameters to search for a hotel
radius = 10000

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    
    filters = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"
    limit =1
    params = {
        "limit": limit,
        "filter": filters,
        "categories": "accommodation.hotel",
        "bias": bias,
        "apiKey": geoapify_key
    }

           
   # print(f"long: {longitude}, lat: {latitude}")
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
   # print(name_address)
   
   
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data

hotel_df

Starting hotel search
faya - nearest hotel: No hotel found
trairi - nearest hotel: Pousada Villa Aurora
guiratinga - nearest hotel: No hotel found
saint-louis - nearest hotel: No hotel found
naberera - nearest hotel: No hotel found
cayenne - nearest hotel: Central Hôtel
banda - nearest hotel: #acnindiafy21
nalut - nearest hotel: No hotel found
rafaela - nearest hotel: Parra Hotel & Suites
itarema - nearest hotel: Pousada Oásis
siavonga - nearest hotel: Lake Safari Lodge Annex
nova olimpia - nearest hotel: Olímpia Hotel
palencia - nearest hotel: No hotel found
kahului - nearest hotel: Maui Seaside Hotel
sao miguel do araguaia - nearest hotel: Hotel São Miguel
aripuana - nearest hotel: No hotel found
sur - nearest hotel: Sur Hotel
chunhuhub - nearest hotel: No hotel found
ouallam - nearest hotel: No hotel found
shakawe - nearest hotel: Drotsky's Cabins
upington - nearest hotel: Die Eiland Holiday Ressort
gwanda - nearest hotel: No hotel found
cabra - nearest hotel: Pensión Guerrero
sao t

,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,No hotel found
12,trairi,BR,-3.2778,-39.2689,51,Pousada Villa Aurora
22,guiratinga,BR,-16.3494,-53.7583,14,No hotel found
29,saint-louis,SN,16.3333,-15.0000,24,No hotel found
34,naberera,TZ,-4.2000,36.9333,54,No hotel found
...,...,...,...,...,...,...
536,nouadhibou,MR,20.9310,-17.0347,27,Hotel valencia
542,bilma,NE,18.6853,12.9164,18,No hotel found
545,kalabo,ZM,-14.9700,22.6814,14,Golden Lodge Luxury Acomodation
550,pali,IN,25.7667,73.3333,26,Hotel Pawan International


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

hotelMap = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    scale = 2,
    color = "City",
    hover_cols = ["Hotel Name","Country","City"]
)

# Display the map
# YOUR CODE HERE
hotelMap

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)